## Imports

In [1]:
import pandas as pd
import numpy as np
import torch
import json

from tqdm import tqdm
from torch import nn
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from timeit import default_timer as timer
from os import walk
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_curve, auc, brier_score_loss

## Data

In [2]:
ROOT_DATA_RAW = './'
HUMAN_JSON_FILE_NAME = 'human.jsonl'
HUMAN_JSON_PATH = f'{ROOT_DATA_RAW}/{HUMAN_JSON_FILE_NAME}'
MODELS_JSON_FOLDER_PATH = f'{ROOT_DATA_RAW}/machines'

In [3]:
BATCH_SIZE = 32
LSTM_UNITS = 256
LSTM_LAYERS = 5
EMBEDDING_SIZE = 512

In [4]:
TEST_SET_FRACTION = 0.3

In [5]:
df = pd.read_json(path_or_buf=HUMAN_JSON_PATH, lines=True)
df['text_index'] = df.index
df['is_llm'] = 0
df['dataset_name'] = Path(HUMAN_JSON_FILE_NAME).stem

In [6]:
dir_path, dir_names, file_names = next(walk(MODELS_JSON_FOLDER_PATH))

for file_name in file_names:
    temp_df = pd.read_json(path_or_buf=f'{MODELS_JSON_FOLDER_PATH}/{file_name}', lines=True)
    temp_df['text_index'] = temp_df.index
    temp_df['is_llm'] = 1
    temp_df['dataset_name'] = Path(file_name).stem

    df = pd.concat([df, temp_df], ignore_index=True)

df.drop(labels=['id'], inplace=True, axis='columns')

In [7]:
df.tail()

,text,text_index,is_llm,dataset_name
5430,Gabby Petito Case: Social Media Detectives and...,1082,1,text-bison-002
5431,Search Intensifies for Missing Woman Gabby Pet...,1083,1,text-bison-002
5432,University of Wisconsin Oshkosh Student Claims...,1084,1,text-bison-002
5433,Gabby Petito Case: The Internet's Role in a Tr...,1085,1,text-bison-002
5434,Gabby Petito Remembered as a 'Super Kind-Heart...,1086,1,text-bison-002


### Tokenize

In [8]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [9]:
df['tokenized_text'] = tokenizer(list(df['text'].to_list()))['input_ids']

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


## Model

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [11]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, layers_num, device, output_size=1, dropout=0):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.layers_num = layers_num
        self.output_size= output_size
        self.dropout = dropout
        self.device = device

        self.embed = nn.Embedding(self.vocab_size, self.embedding_size, device=self.device)

        self.lstm = nn.LSTM(
            input_size=self.embedding_size,
            hidden_size=self.hidden_size,
            num_layers=self.layers_num,
            batch_first=True,
            dropout=self.dropout,
            device=self.device
        )

        self.fc = nn.Linear(
            self.hidden_size,
            self.output_size
        )

    def forward(self, X, lengths):
        embeddings = self.embed(X)

        seq_output, (h_n, c_n) = self.lstm(embeddings)

        out = seq_output.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        logits = self.fc(out)
        return logits

## Dataset

In [12]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.size

    def __getitem__(self, index):
        return (
            self.X[index],
            self.y[index]
        )

In [13]:
def collate_fn(batch):
  # We want to sort the batch by seq length,
  # in order to make the computation more efficient
  batch = sorted(batch, key=lambda x: len(x[0]), reverse=True)

  inputs = [torch.LongTensor(x[0]).to(device) for x in batch]
  padded_input = nn.utils.rnn.pad_sequence(inputs, batch_first=True)

  lengths = torch.LongTensor([len(x[0]) for x in batch]).to(device)

  y = torch.FloatTensor(np.array([x[1] for x in batch])).reshape(-1, 1).to(device)

  return padded_input, lengths, y

## Train and test functions

In [20]:
def calculate_accuracy(y_true, y_hat):
    correct_pred = torch.eq(torch.sigmoid(y_hat).round(), y_true).sum().item()
    return (correct_pred / len(y_hat)) * 100

def calculate_f1(y_true, y_hat):
    y_pred = torch.sigmoid(y_hat).round()
    return f1_score(y_true, y_pred)

def calculate_brier(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)
    return brier_score_loss(y_true, y_prob)

def calculate_auc(y_true, y_hat):
    y_prob = torch.sigmoid(y_hat)

    false_positive_rates, true_positive_rates, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(false_positive_rates, true_positive_rates)

    return roc_auc, false_positive_rates, true_positive_rates

def train_step(model, dataloader, loss_fn, optimizer, device):

    model.train()

    train_loss, train_acc = 0, 0
    steps = 0

    for X, lengths, y in dataloader:
        X, y = X.to(device), y.to(device)

        y_hat = model(X, lengths)

        loss = loss_fn(y_hat, y)
        train_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += calculate_accuracy(y_true=y, y_hat=y_hat)
        steps += 1

    return train_loss / steps, train_acc / steps


def test_step(model, dataloader, loss_fn, device):

    model.eval()

    all_y_true = []
    all_y_hat = []

    test_loss = 0
    steps = 0

    with torch.inference_mode():
        for X, lengths, y in dataloader:

            X, y = X.to(device), y.to(device)

            y_hat = model(X, lengths)

            all_y_true.extend(y)
            all_y_hat.extend(y_hat)

            loss = loss_fn(y_hat, y)
            test_loss += loss.item()

            steps += 1

        all_y_true = torch.FloatTensor(all_y_true)
        all_y_hat = torch.FloatTensor(all_y_hat)

        test_accuracy = calculate_accuracy(all_y_true, all_y_hat)
        test_f1 = calculate_f1(all_y_true, all_y_hat)
        test_brier = calculate_brier(all_y_true, all_y_hat)
        test_auc_tuple = calculate_auc(all_y_true, all_y_hat)

    return test_loss / steps, test_accuracy, test_f1, test_brier, test_auc_tuple

def train(model,
          train_dataloader,
          test_dataloader,
          optimizer,
          loss_fn,
          epochs,
          device):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
        "test_f1": [],
        "test_brier": [],
        "test_auc_tuple": []
    }

    model.to(device)

    for epoch in tqdm(range(epochs)):

        start_time = timer()
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        end_time = timer()

        test_loss, test_acc, test_f1, test_brier, test_auc_tuple = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
        results["test_f1"].append(test_f1)
        results["test_brier"].append(test_brier),
        results["test_auc_tuple"].append(test_auc_tuple)

        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f} | "
            f"test_f1: {test_f1:.4f} | "
            f"test_brier: {test_brier:.4f} | "
            f"time: {(end_time-start_time):.4f}"
        )

    return results

## Train on each LLM dataset separately and test against all others

In [ ]:
def test_against_all(model, trained_llm_name, human_test_df, df, loss_fn, device):
    all_results = []

    for dataset_name in df['dataset_name'].unique():
        if dataset_name in [trained_llm_name, Path(HUMAN_JSON_FILE_NAME).stem]:
            continue

        llm_df = df.loc[df['dataset_name'] == dataset_name]
        test_df = pd.concat([human_test_df, llm_df], ignore_index=True)

        test_dataset = TextDataset(test_df['tokenized_text'], test_df['is_llm'])
        test_dataloader = DataLoader(
            test_dataset,
            batch_size=BATCH_SIZE,
            shuffle=False,
            drop_last=False,
            collate_fn=collate_fn
        )

        results = test_step(
            model,
            test_dataloader,
            loss_fn,
            device
        )

        all_results.append({
            dataset_name: results
        })

    return all_results

In [18]:
human_df = df.loc[df['is_llm'] == 0]
human_train_df, human_test_df = train_test_split(human_df, test_size=TEST_SET_FRACTION, random_state=69)

In [21]:
final_results = []

for llm_name in df['dataset_name'].unique():
    if llm_name == Path(HUMAN_JSON_FILE_NAME).stem:
        continue

    llm_df = df.loc[df['dataset_name'] == llm_name]
    llm_train_df, llm_test_df = train_test_split(llm_df, test_size=TEST_SET_FRACTION, random_state=69)

    train_df = pd.concat([human_train_df, llm_train_df], ignore_index=True)
    test_df = pd.concat([human_test_df, llm_test_df], ignore_index=True)

    train_dataset = TextDataset(train_df['tokenized_text'], train_df['is_llm'])
    test_dataset = TextDataset(test_df['tokenized_text'], test_df['is_llm'])

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        drop_last=False,
        collate_fn=collate_fn
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        drop_last=False,
        collate_fn=collate_fn
    )

    model = RNN(tokenizer.vocab_size, EMBEDDING_SIZE, LSTM_UNITS, LSTM_LAYERS, device, dropout=0.6).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00008)
    loss_fn = nn.BCEWithLogitsLoss()

    print(f'Training model for {llm_name}...')

    current_results = train(
        model,
        train_dataloader,
        test_dataloader,
        optimizer,
        loss_fn,
        epochs=10,
        device=device
    )    

    print(f'Finished training model for {llm_name}')

    print(f'Testing against all for {llm_name}...')
    
    results_against_all = test_against_all(
        model,
        llm_name,
        human_test_df,
        df,
        loss_fn,
        device
    )

    final_results.append({
        'results_against_itself': current_results,
        'results_against_all': results_against_all
    })

    print(f'Finished testing against all for {llm_name}')

Training model for vicgalle-gpt2-open-instruct-v1...


 10%|█         | 1/10 [00:16<02:25, 16.22s/it]

Epoch: 1 | train_loss: 0.6938 | train_acc: 50.1302 | test_loss: 0.6969 | test_acc: 50.0000 | test_f1: 0.6667 | test_brier: 0.2521 | time: 14.4865


 20%|██        | 2/10 [00:31<02:05, 15.69s/it]

Epoch: 2 | train_loss: 0.6933 | train_acc: 49.7396 | test_loss: 0.6984 | test_acc: 50.0000 | test_f1: 0.6667 | test_brier: 0.2526 | time: 13.5820


...

Epoch: 7 | train_loss: 0.1718 | train_acc: 96.0938 | test_loss: 0.1085 | test_acc: 95.7187 | test_f1: 0.9581 | test_brier: 0.0322 | time: 13.8549


 80%|████████  | 8/10 [02:11<00:32, 16.08s/it]

Epoch: 8 | train_loss: 0.1603 | train_acc: 94.8568 | test_loss: 0.1763 | test_acc: 91.8960 | test_f1: 0.9250 | test_brier: 0.0547 | time: 14.1863


 80%|████████  | 8/10 [02:13<00:33, 16.63s/it]


KeyboardInterrupt: 